Computes the necessary files for the three Datawrapper data visualization pieces.

In [1]:
from babel.dates import format_date
from datetime import datetime, timedelta
import pandas as  pd

#### Reading data

In [2]:
df = pd.read_csv("../output/city-recent-daily-means.csv")

In [3]:
df = df.set_index(pd.to_datetime(df.date))

In [4]:
df = df.rename(columns={'mean':'daily_mean'})

#### Detects the months needed and merges

In [5]:
# Adds the month
df['month'] = df.index.month

In [6]:
# Gets the latest month
latest_month = df['month'].idxmax().month

In [7]:
# Keeps only the latest month
df = df[df['month']==latest_month]

In [8]:
# Reads the associated data
month_history = pd.read_csv(f"../output/city-monthly-means/City_Month_{latest_month}_Mean")
month_history = month_history.rename(columns={"mean":"historical_mean"})

In [9]:
# Adds historical mean
df = df.merge(month_history)

In [10]:
# Converts from kg/m3 to microgram/m3
df['daily_mean'], df['historical_mean'] = df['daily_mean'] * 1e9, df['historical_mean'] * 1e9

In [11]:
# Gets the variation from the mean
df['anomaly'] = df['daily_mean'] / df['historical_mean']

In [12]:
df.head()

,CTR_MN_ISO,CTR_MN_NM,ID_HDC_G0,P15,UC_NM_LST,UC_NM_MN,date,daily_mean,month,lat,lon,historical_mean,anomaly
0,ARG,Argentina,962,441985.894928,San Juan,San Juan,2024-09-01,5.152734,9,-31.536676,-68.542058,5.142087,1.002071
1,ARG,Argentina,962,441985.894928,San Juan,San Juan,2024-09-02,17.472118,9,-31.536676,-68.542058,5.142087,3.397865
2,ARG,Argentina,962,441985.894928,San Juan,San Juan,2024-09-03,3.206359,9,-31.536676,-68.542058,5.142087,0.623552
3,ARG,Argentina,962,441985.894928,San Juan,San Juan,2024-09-04,5.140476,9,-31.536676,-68.542058,5.142087,0.999687
4,ARG,Argentina,962,441985.894928,San Juan,San Juan,2024-09-05,7.467998,9,-31.536676,-68.542058,5.142087,1.452328


#### Selecting cities

In [13]:
# Brazilian capitals, with the correct correspondence
br_cities =  {
    'Porto Alegre': 'Porto Alegre (RS)',
    'Florianopolis': 'Florianópolis (SC)',
    'Curitiba': 'Curitiba (PR)',
    'Sao Paulo': 'São Paulo (SP)',
    'Belo Horizonte': 'Belo Horizonte (MG)',
    'Rio de Janeiro': 'Rio de Janeiro (RJ)',
    'Vila Velha': 'Vitória (ES)', # Dado leva o nome de Vila Velha, mas é da região metropolitana de Vitória
    'Cuiaba': 'Cuiabá (MT)',
    'Campo Grande': 'Campo Grande (MS)',
    'Brasilia': 'Brasília (DF)',
    'Goiania': 'Goiânia (GO)',
    'Palmas': 'Palmas (TO)',
    'Manaus': 'Manaus (AM)',
    'Belem': 'Belém (PA)',
    'Rio Branco': 'Rio Branco (AC)',
    'Boa Vista': 'Boa Vista (RR)',
    'Macapa': 'Macapá (AP)',
    'Porto Velho': 'Porto Velho (RO)',
    'Sao Luis': 'São Luís (MA)',
    'Teresina': 'Teresina (PI)',
    'Natal': 'Natal (RN)',
    'Fortaleza': 'Fortaleza (CE)',
    'Recife': 'Recife (PE)',
    'Joao Pessoa': 'João Pessoa (PB)',
    'Aracaju': 'Aracaju (SE)',
    'Maceio': 'Maceió (AL)',
    'Salvador': 'Salvador (BA)'
}

In [14]:
# The Brazilian charts will derive from here
df_br = df[df.UC_NM_MN.isin(br_cities.keys())]
df_br['UC_NM_MN'] = df_br['UC_NM_MN'].replace(br_cities)

/var/folders/yl/bd_xy2c53495c2z2g0w0yrzdtddnbq/T/ipykernel_59656/32914145.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_br['UC_NM_MN'] = df_br['UC_NM_MN'].replace(br_cities)


In [15]:
# The South American charts will derive from here
# TO DO

In [16]:
df_br['datetime'] = pd.to_datetime(df.date)

/var/folders/yl/bd_xy2c53495c2z2g0w0yrzdtddnbq/T/ipykernel_59656/155995396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_br['datetime'] = pd.to_datetime(df.date)


#### Multiline chart

In [17]:
# Datawrapper forma
dw_multiline_br = df_br.pivot(index='date', columns=['UC_NM_MN'], values='anomaly')

In [18]:
# Show in percentage points of total
dw_multiline_br = dw_multiline_br * 100

In [19]:
dw_multiline_br['baseline'] = 100

In [36]:
# Date should be its own column
dw_multiline_br = dw_multiline_br.reset_index()

In [37]:
dw_multiline_br.to_csv("../output/datawrapper-multiline-br.csv", index=False)

#### Pollution stripes

In [21]:
dw_stripes_br = df_br.pivot(index='UC_NM_MN', columns='date', values='daily_mean').reset_index()

In [22]:
# Adds the necessary extra row for information
dw_stripes_br.loc[-1] = ""  # adding a row
dw_stripes_br.index = dw_stripes_br.index + 1  # shifting index
dw_stripes_br = dw_stripes_br.sort_index()  # sorting by index

In [23]:
# Insert new column in specific position
dw_stripes_br.insert(loc=1, column="label", value="")

In [24]:
# Adds other informative values
min_date = pd.to_datetime(dw_stripes_br.columns[2]).strftime("%d/%m")
max_date = pd.to_datetime(dw_stripes_br.columns[-1]).strftime("%d/%m")

dw_stripes_br.loc[0, 'label'] = f'{min_date} – {max_date}'
dw_stripes_br.loc[0, 'UC_NM_MN'] = 'Cidade'

In [25]:
dw_stripes_br

date,UC_NM_MN,label,2024-09-01,2024-09-02,2024-09-03,2024-09-04,2024-09-05,2024-09-06,2024-09-07,2024-09-08,2024-09-09,2024-09-10,2024-09-11
0,Cidade,01/09 – 11/09,,,,,,,,,,,
1,Aracaju (SE),,5.774878,12.178995,12.456645,10.257351,10.224584,9.72766,7.914113,8.905372,7.934762,9.100605,7.231578
2,Belo Horizonte (MG),,8.321189,17.032488,29.857136,34.032404,30.718047,28.772413,15.861812,21.585042,15.395418,13.896411,5.986929
3,Belém (PA),,9.29731,9.578344,12.178804,7.348999,7.369967,6.642637,7.876628,9.408293,8.798545,9.391717,8.381199
4,Boa Vista (RR),,10.819026,9.894495,9.488356,8.331139,12.441079,9.896644,11.991467,12.005633,10.34759,12.556366,17.634055
5,Brasília (DF),,4.597651,7.168618,8.117225,7.808461,24.568676,25.151049,13.586624,10.974288,10.590771,8.393132,5.111564
6,Campo Grande (MS),,18.824624,33.797622,6.123705,7.747383,65.230687,29.08806,28.755817,15.739145,24.726408,23.310159,26.374575
7,Cuiabá (MT),,37.136315,33.126155,15.595948,22.578758,48.176884,59.752337,28.985016,17.099894,17.975005,28.973585,38.562612
8,Curitiba (PR),,21.066591,24.271822,18.423077,21.333846,32.248418,17.950673,30.237421,37.035728,35.383545,40.896873,26.438531
9,Florianópolis (SC),,20.302273,9.750763,10.665398,22.126254,12.407707,8.747425,11.016971,27.540966,17.062724,20.791775,24.112977


In [26]:
dw_stripes_br.to_csv("../output/datawrapper-stripes-br.csv", index=False)

#### Map

In [27]:
# Geet only the desired columns
dw_map_br = df_br[['UC_NM_MN', 'CTR_MN_NM', 'daily_mean', 'date', 'datetime', 'lat', 'lon']]

In [28]:
# Keeps only the last seven days
max_date = dw_map_br.datetime.max()
week_ago = max_date - timedelta(days=7)

dw_map_br = dw_map_br[(dw_map_br.datetime <= max_date) & (dw_map_br.datetime > week_ago)]

In [29]:
dw_map_br = dw_map_br.groupby('UC_NM_MN')['daily_mean'].mean().reset_index()

In [30]:
start, end = week_ago.strftime('%d/%m'), max_date.strftime('%d/%m')
time_interval = f"Média semanal de partículas PM 2.5 por metro cúbico entre {start} e {end}"
time_interval

'Média semanal de partículas PM 2.5 por metro cúbico entre 04/09 e 11/09'

In [31]:
dw_map_br = dw_map_br.rename(columns={'daily_mean':time_interval})

In [32]:
dw_map_br.head()

,UC_NM_MN,Média semanal de partículas PM 2.5 por metro cúbico entre 04/09 e 11/09
0,Aracaju (SE),8.719811
1,Belo Horizonte (MG),18.888010
2,Belém (PA),8.266998
3,Boa Vista (RR),12.410405
4,Brasília (DF),14.053729


In [33]:
dw_map_br.to_csv("../output/datwrapper-map-br.csv", index=False)